In [1]:
configs = {
  "fs.adl.oauth2.access.token.provider.type": "ClientCredential",
  "fs.adl.oauth2.client.id": "4ab956bb-5a0b-45b3-a87b-22424b89ea36",
  "fs.adl.oauth2.credential": ".2SD44SJ2nti=QtOym[@anQ[cXiW?6v.",
  "fs.adl.oauth2.refresh.url": "https://login.microsoftonline.com/5d7e4366-1b9b-45cf-8e79-b14b27df46e1/oauth2/token"}

dbutils.fs.mount(
  source = "adl://imagerepo.azuredatalakestore.net/dogImages",
  mount_point  = "/mnt/dogImages",
  extra_configs = configs)

dbutils.fs.mount(
  source = "adl://imagerepo.azuredatalakestore.net/cifar-10-batches-py",
  mount_point  = "/mnt/cifar-10-batches-py",
  extra_configs = configs)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-949048213614856> in <module> 
 8 source = "adl://imagerepo.azuredatalakestore.net/dogImages" , 
 9 mount_point = "/mnt/dogImages" , 
 ---> 10 extra_configs = configs)
 11 
 12 dbutils.fs.mount(

 /local_disk0/tmp/1587916602159-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o222.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/dogImages; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/dogImages
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/dogImages
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:224)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:326)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:220)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:304)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:282)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:52)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:79)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:79)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:48)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:428)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.ServerBackend.withAttributio

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
[str(name).upper() for name in unpickle("/dbfs/mnt/cifar-10-batches-py/batches.meta")[b'label_names']]

Out[2]: ["B'AIRPLANE'",
 "B'AUTOMOBILE'",
 "B'BIRD'",
 "B'CAT'",
 "B'DEER'",
 "B'DOG'",
 "B'FROG'",
 "B'HORSE'",
 "B'SHIP'",
 "B'TRUCK'"]

In [4]:
import numpy as np
from keras.utils import np_utils
# prepare data
classfication_names = unpickle("/dbfs/mnt/cifar-10-batches-py/batches.meta")[b'label_names']
test_data = unpickle("/dbfs/mnt/cifar-10-batches-py/test_batch")
test_tensors = np.array([np.moveaxis(test_data[b'data'][i].reshape(-1,32,32), 0, -1) for i in range(test_data[b'data'].shape[0])])
test_targets = np_utils.to_categorical(np.array(test_data[b'labels']), 20)

train_data = unpickle("/dbfs/mnt/cifar-10-batches-py/data_batch_5")
valid_tensors = np.array([np.moveaxis(train_data[b'data'][i].reshape(-1,32,32), 0, -1) for i in range(train_data[b'data'].shape[0])])
valid_targets = np_utils.to_categorical(np.array(train_data[b'labels']), 10)
train_tensors = valid_tensors
train_targets = valid_targets
for i in range(1,5):
  train_data = unpickle("/dbfs/mnt/cifar-10-batches-py/data_batch_" + str(i))
  tensors = np.array([np.moveaxis(train_data[b'data'][i].reshape(-1,32,32), 0, -1) for i in range(train_data[b'data'].shape[0])])
  targets = np_utils.to_categorical(np.array(train_data[b'labels']), 10)
  train_tensors = np.concatenate((train_tensors, tensors), axis=0)
  train_targets = np.concatenate((train_targets, targets), axis=0)
train_tensors = train_tensors[10000:]
train_targets = train_targets[10000:]

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping

In [5]:
def extract_VGG16(tensor):
	from keras.applications.vgg16 import VGG16, preprocess_input
	return VGG16(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_VGG19(tensor):
	from keras.applications.vgg19 import VGG19, preprocess_input
	return VGG19(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_Resnet50(tensor):
	from keras.applications.resnet50 import ResNet50, preprocess_input
	return ResNet50(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_Xception(tensor):
	from keras.applications.xception import Xception, preprocess_input
	return Xception(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_InceptionV3(tensor):
	from keras.applications.inception_v3 import InceptionV3, preprocess_input
	return InceptionV3(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

In [6]:
train_ResNet50 = extract_Resnet50(train_tensors)
valid_ResNet50 = extract_Resnet50(valid_tensors)
test_ResNet50 = extract_Resnet50(test_tensors)

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4479: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:203: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2041: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

/databricks/python/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
 warnings.warn('The output shape of `ResNet50(include_top=False)` '
Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

 8192/94653016 [..............................] - ETA: 3s
 2990080/94653016 [..............................] - ETA: 1s
11436032/94653016 [==>...........................] - ETA: 0s
16515072/94653016 [====>.........................] - ETA: 0s
21536768/94653016 [=====>........................] - ETA: 0s
26607616/94653016 [=======>......................] - ETA: 0s
31719424/94653016 [=========>....................] - ETA: 0s
36741120/94653016 [==========>...................] - ETA: 0s
41623552/94653016 [============>.................] - ETA: 0s
47144960/94653016 [=============>................] - ETA: 0s
52125696/94653016 [===============>..............] - ETA: 0s
57204736/94653016 [=================>............] - ETA: 0s
62210048/94653016 [==================>...........] - ETA: 0s
67280896/94653016 [====================>.........] - ETA: 0s
72294400/94653016 [=====================>........] - ETA: 0s
77307904/94653016 [=======================>......] - ETA: 0s
82370560/94653016 [=========================>....] - ETA: 0s
87416832/94653016 [==========================>...] - ETA: 0s
92405760/94653016 [=======

In [7]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Conv1D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

my_ResNet50_model = Sequential()
# my_ResNet50_model.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
my_ResNet50_model.add(Dense(32, input_shape=train_ResNet50.shape[1:], activation='relu'))
my_ResNet50_model.add(Flatten())
my_ResNet50_model.add(Dense(10, activation='softmax'))

my_ResNet50_model.summary()

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

Model: "sequential_1"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
dense_1 (Dense) (None, 1, 1, 32) 65568 
_________________________________________________________________
flatten_1 (Flatten) (None, 32) 0 
_________________________________________________________________
dense_2 (Dense) (None, 10) 330 
=================================================================
Total params: 65,898
Trainable params: 65,898
Non-trainable params: 0
_________________________________________________________________

In [8]:
my_ResNet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.

In [9]:
from keras.callbacks import ModelCheckpoint


checkpointer = ModelCheckpoint(filepath='weights.best.CifarResNet-50.hdf5', 
                               verbose=1, save_best_only=True)

my_ResNet50_model.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=20, batch_size=50, callbacks=[checkpointer], verbose=1)

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

Train on 40000 samples, validate on 10000 samples
Epoch 1/20

 50/40000 [..............................] - ETA: 35:33 - loss: 3.9338 - acc: 0.0800
 900/40000 [..............................] - ETA: 1:58 - loss: 2.3553 - acc: 0.2922 
 1750/40000 [>.............................] - ETA: 1:00 - loss: 2.0809 - acc: 0.3417
 2650/40000 [>.............................] - ETA: 39s - loss: 1.9343 - acc: 0.3785 
 3500/40000 [=>............................] - ETA: 30s - loss: 1.8654 - acc: 0.3926
 4350/40000 [==>...........................] - ETA: 24s - loss: 1.8115 - acc: 0.4032
 5150/40000 [==>...........................] - ETA: 20s - loss: 1.7505 - acc: 0.4231
 6050/40000 [===>..........................] - ETA: 17s - loss: 1.7127 - acc: 0.4322
 6950/40000 [====>.........................] - ETA: 14s - loss: 1.6661 - acc: 0.4455
 7850/40000 [====>.........................] - ETA: 12s - loss: 1.6362 - acc: 0.4581
 8750/40000 [=====>........................] - ETA: 11s - loss: 1.6089 - acc: 0.4685
 9400/40000 [======>.......................] - ETA: 10s - loss: 1.5890 - acc: 0.4760
10250/40000 [======>.......................] - ETA: 9s - loss: 1.5665 - acc: 0.4828 
11050/40000 [=======>......................] - ETA: 8s - loss: 1.5523 - acc: 0.4878
11950/40000 [=======>......................] - ETA: 7s - loss: 1.5363 - acc: 0.4935
12850/40000 [========>.....................] - ETA: 7s - loss: 1.5219 - acc: 0.4982
13750/40000 [=========>....................] - ETA: 6s - loss: 1.5074 - acc: 0.5026
14650/40000 [=========>....................] - ETA: 6s - loss: 1.4920 - acc: 0.5068
15550/40000 [==========>...................] - ETA: 5s - loss: 1.4773 - acc: 0.5116
16450/40000 [===========>..................] - ETA: 5s - loss: 1.4661 - acc: 0.5152
17250/40000 [===========>..................] - ETA: 4s - loss: 1.4546 - acc: 0.5182
18000/40000 [============>.................] - ETA: 4s - loss: 1.4452 - acc: 0.5206
18750/40000 [=============>................] - ETA: 4s - loss: 1.4373 - acc: 0.5228
19600/40000 [=============>................] - ETA: 4s - loss: 1.4251 - acc: 0.5268
20500/40000 [==============>...............] - ETA: 3s - loss: 1.4157 - acc: 0.5297
21400/40000 [===============>..............] - ETA: 3s - loss: 1.4068 - acc: 0.5323
22300/40000 [===============>..............] - ETA: 3s - loss: 1.3981 - acc: 0.5355
23200/40000 [================>.............] - ETA: 2s - loss: 1.3893 - acc: 0.5383
24000/40000 [=================>............] - ETA: 2s - loss: 1.3826 - acc: 0.5401
24900/40000 [=================>............] - ETA: 2s - loss: 1.3776 - acc: 0.5416
25800/40000 [==================>...........] - ETA: 2s - loss: 1.3705 - acc: 0.5447
26700/40000 [===================>..........] - ETA: 2s - loss: 1.3645 - acc: 0.5462
27650/40000 [===================>..........] - ETA: 1s - loss: 1.3552 - acc: 0.5484
28500/40000 [====================>.........] - ETA: 1s - loss: 1.3503 - acc: 0.5499
29400/40000 [=====================>........] - ETA: 1s - loss: 1.3464 - acc: 0.5511
30300/40000 [=====================>........] - ETA: 1s - loss: 1.3387 - acc: 0.5536
31200/40000 [======================>.......] - ETA: 1s - loss: 1.3352 - acc: 0.5553
31850/40000 [======================>.......] - ETA: 1s - loss: 1.3324 - acc: 0.5560
32800/40000 [======================

In [10]:
my_ResNet50_model.load_weights('weights.best.CifarResNet-50.hdf5')

In [11]:
my_ResNet50_predictions = [np.argmax(my_ResNet50_model.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(my_ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(my_ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 64.2600%

In [12]:
import mlflow.keras
mlflow.keras.save_model(my_ResNet50_model, '/dbfs/mnt/cifar-10-batches-py/saved_model/cifar10_resnet50')

In [13]:
import requests
import json
img = open('/dbfs/mnt/dogImages/dogImages/test/134.Shiba_inu/Capture.PNG', 'rb').read()
headers = {'Content-Type':'application/json'}

resp = requests.post('http://1f770b22-8329-4656-b1c6-5ac11e462c30.eastus.azurecontainer.io/score',data=img, headers=headers, timeout=300)
json.loads(resp.json())['predicted_label']

Out[13]: 'MEDIUM_MAMMALS'

In [14]:
test_ResNet50

Out[14]: array([[[[-0. , -0. , -0. , ..., -0. ,
 -0. , -0. ]]],


 [[[-0. , -0. , -0. , ..., -0. ,
 -0. , -0. ]]],


 [[[-0. , -0. , -0. , ..., -0. ,
 -0. , -0. ]]],


 ...,


 [[[-0. , -0. , -0. , ..., -0. ,
 -0. , -0. ]]],


 [[[-0. , -0. , 7.638767 , ..., 5.11226 ,
 -0. , -0. ]]],


 [[[-0. , -0. , -0. , ..., 0.8227267,
 -0. , -0. ]]]], dtype=float32)